In [ ]:
import pandas as pd

# Leer el archivo CSV de titulados ed_sup 2022
part1 = pd.read_csv('part1.csv')
part2 = pd.read_csv('part2.csv')
part3 = pd.read_csv('part3.csv')
part4 = pd.read_csv('part4.csv')
df_titulados = pd.concat([part1, part2, part3, part4])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_titulados.head()

In [ ]:
df_notas2015 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2015_WEB.csv',sep=";", encoding='latin1')
df_notas2016 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2016_WEB.csv',sep=";", encoding='latin1')
df_notas2017 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2017_WEB.csv',sep=";", encoding='latin1')
df_notas2018 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2018_WEB.csv',sep=";", encoding='latin1')
df_notas2019 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2019_WEB.csv',sep=";", encoding='latin1')
df_notas2020 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2020_WEB.csv',sep=";", encoding='latin1')
df_notas2015_2020 = pd.concat([df_notas2015, df_notas2016, df_notas2017, df_notas2018, df_notas2019, df_notas2020])


In [ ]:
df_notas2015_2020.head()

Proposicion preprocesado

In [ ]:
#vemos el tamaño para saber las dimensiones en futuras referencias
df_titulados.shape

In [ ]:
#evaluamos las columnas para ver que utilizaremos
df_titulados.head(20)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['fec_nac_alu','anio_ing_carr_ori','sem_ing_carr_ori','sem_ing_carr_act','fecha_obtencion_titulo','cod_inst','cod_sede','cod_carrera','dur_proceso_tit','provincia_sede','comuna_sede','version','tipo_plan_carr','area_cineunesco','area_cine_f_97','subarea_cine_f_97','area_cine_f_13','subarea_cine_f_13','area_carrera_generica_n']
df_titulados.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_titulados.isnull().sum()

In [ ]:
#Notamos que mrun tiene nulos lo cual no lo consideraremos ya que son necesarios para un merge a futuro que haremos
df_titulados.dropna(subset=["mrun"], inplace=True)

In [ ]:
#tambien notamos que hay muchos nulos en nombres de itulos obtenidos, lo que asumimos que no lo consiguieron.
# por lo que lo rellenaremos como no completados
df_titulados["nomb_titulo_obtenido"].fillna('No completado', inplace=True)

In [ ]:
#lo mismo para el nombre de grado obtenido
df_titulados["nomb_grado_obtenido"].fillna('Grado superior no obtenido', inplace=True)

In [ ]:
#comprobamos si hay duplicados
df_titulados.duplicated().sum()
df_titulados.drop_duplicates

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
# (veremos en el codigo mas adelante si son necesarios)
df_titulados.info()

Preprocesado notas

In [ ]:
df_notas2015_2020.shape

In [ ]:
df_notas2015_2020.head(2)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['DGV_RBD','COD_REG_RBD','RBD','NOM_REG_RBD_A','COD_PRO_RBD','COD_COM_RBD','NOM_COM_RBD','COD_DEPROV_RBD','NOM_DEPROV_RBD']
df_notas2015_2020.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_notas2015_2020.isnull().sum()

In [ ]:
#comprobamos si hay duplicados
df_notas2015_2020.duplicated().sum()

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
# (veremos en el codigo mas adelante si son necesarios)
df_titulados.info()